<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/Chatbot_%2B_Memory_%2B_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q langchain_groq gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00


In [2]:
from langchain_groq import ChatGroq
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))

In [3]:
import json
import datetime
def submit_complaint(complaint, name):
    issues = []
    try:
        with open('issues.json', 'r') as f:
            issues = json.load(f)
    except FileNotFoundError:
        pass  # If the file doesn't exist, we'll create it below

    # Adding the new issue
    issues.append({'name': name, 'issue': complaint, 'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')})

    # Saving the updated issues list
    with open('issues.json', 'w') as f:
        json.dump(issues, f, indent=4)

    return json.dumps({"success": "Issue submitted successfully."})

def calculate(expression):
    try:
        # Prepare a local namespace to safely execute the expression
        local_namespace = {}

        # Execute the expression using exec
        exec(expression, {}, local_namespace)

        # Return the result if it's stored in a specific variable
        return local_namespace.get('result', "No result found")

    except Exception as e:
        return f"Error in calculation: {e}"

In [4]:
class Chatbot:
    prompt = """
    You operate in a structured loop consisting of the following steps: Thought, Action, PAUSE, and Observation. At the end of this loop, you will output an Answer. Follow this process meticulously:

	1.	Thought: Use this step to articulate your reasoning about the question or task you’ve been asked to handle. Think through the problem and decide which action to take next.
	2.	Action: Choose and execute one of the available actions based on your Thought. Once you initiate an action, return PAUSE to wait for further instructions.
	3.	Observation: This will contain the outcome or result of the Action you took. Use this information to inform your next step.
	4.	Answer: After sufficient loops of Thought, Action, PAUSE, and Observation, when the task is completed, provide a final Answer to the user.

Available Actions:

	•	calculate:
	•	Usage: calculate:
	•	Parameter: expression — a mathematical expression in Python.
	•	Function: Evaluates the expression and returns the result. Make sure to use floating-point syntax if necessary.
  •	import math if it uses math functions. make sure the expression returns something like result = math.log(2 * math.sqrt(36))
	•	submit_complaint:
	•	Usage: submit_complaint
	•	Parameters:
	•	complaint_text (str) — The content of the complaint.
	•	person_name (str) — The name of the person submitting the complaint.
	•	Function: Submits the complaint. If person_name is missing, ask for the person’s name and return PAUSE.

Process Guidelines:

	•	If a user requests to submit a complaint but doesn’t provide their name, ask for their name, then return PAUSE.
	•	If the request or question does not align with any of the available actions, proceed with a normal conversation.

Example Session:

Question: Submit a complaint about Microsoft Office 365. My name is Balaji

Thought: I should submit a complaint about Microsoft Office 365 with the person’s name, Balaji.

Action: submit_complaint | complaint_text: Microsoft Office 365 | person_name: Balaji

PAUSE

On the next call, you will receive an Observation:

Observation: The issue is submitted successfully.

Answer: Your complaint about Microsoft Office 365 has been submitted successfully, Balaji.

Important Notes:

	•	Follow the loop of Thought → Action → PAUSE → Observation until the task is completed.
	•	Only provide the final Answer after processing all necessary steps.
	•	If the situation doesn’t match any of the defined actions, handle it as a normal conversation.

            """

    def __init__(self):
      self.messages = [] # setting up a basic memory
      self.messages.append({"role": "system", "content": self.prompt})

    def __call__(self, user_message):
      self.messages.append({"role": "user", "content": user_message})
      ai_message = llm_groq.invoke(self.messages)
      self.messages.append({"role": "assistant", "content": ai_message.content})
      return ai_message.content

In [5]:
bot = Chatbot()
response = bot("submit a complaint about Microsoft Office 365. My name is Balaji")
response

"Thought: I should submit a complaint about Microsoft Office 365 with the person's name, Balaji.\n\nAction: submit_complaint | complaint_text: Microsoft Office 365 | person_name: Balaji\n\nPAUSE"

In [6]:
bot = Chatbot()
response = bot("I don't know what to do in life. HElp me with some guidance.")
response

"Thought: It seems like the user is seeking guidance and direction in life. I need to provide a supportive and non-judgmental response to help them feel heard and understood.\n\nAction: I'll respond with a empathetic message to acknowledge their feelings and encourage them to share more about their concerns.\n\nOutput: I totally understand how overwhelming it can feel when you're unsure about the direction of your life. It takes a lot of courage to admit that, and I'm here to listen and help in any way I can. Could you tell me more about what's been going on that's making you feel this way?\n\nPAUSE"

In [7]:
response = bot("find log of 2 times square root of 36")
response

'Thought: The user wants me to calculate the logarithm of 2 times the square root of 36. I need to use the calculate action to evaluate this mathematical expression.\n\nAction: calculate | expression: import math; result = math.log(2 * math.sqrt(36))\n\nPAUSE'

In [8]:
import re
# Extract the Action part
def handle_action(response):
    action_match = re.search(r'Action: ([^\n]+)', response)
    action = action_match.group(1).strip() if action_match else None

    # Initialize empty parameters dictionary
    params = {}

    if action:
        # Check if it's a 'calculate' action
        if action.startswith("calculate:"):
            # Extract the full calculation expression after 'calculate:'
            calculation_expression = action.split("calculate:", 1)[1].strip()
            params['expression'] = calculation_expression

        # If action includes parameters (like in a complaint), split by '|'
        elif '|' in action:
            parts = action.split('|')
            action = parts[0].strip()

            for part in parts[1:]:
                key_value = part.split(':', 1)  # Split only on the first ':'
                if len(key_value) == 2:
                    params[key_value[0].strip()] = key_value[1].strip()

    print(f"Action: {action}")
    print(f"Parameters: {params}")

    if action == "submit_complaint":
        complaint = params.get('complaint_text')
        name = params.get('person_name')
        if complaint and name:
            action_response = submit_complaint(complaint=complaint, name=name)
            return response + "\n" + action_response
        else:
            return response + "\n" + "Error: Missing complaint_text or person_name"

    elif action.startswith("calculate"):
        expression = params.get('expression')
        action_response = calculate(expression=expression)
        return response + "\n" + str(action_response)

    else:
        return response  # there is no action here, return the original response

In [ ]:
import gradio as gr
bot = Chatbot()
def chat(message, history):
    response = bot(message)
    return handle_action(response)

demo = gr.ChatInterface(
    fn=chat,
    title="Simple Chatbot",
    description="This is a chatbot built as part of Modern AI Pro Essentials program",
)
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://131df8528ea844009c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Action: calculate
Parameters: {'expression': '343 ** (1/3)'}
Action: calculate
Parameters: {'expression': 'import math; result = math.sqrt(25)'}
Action: calculate
Parameters: {'expression': '56 ** 2'}
Action: submit_complaint
Parameters: {}
Action: submit_complaint
Parameters: {'complaint_text': 'Internet connection has been down for the past three days', 'person_name': 'Vijayamurugan'}
